In [1]:
import numpy as np
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense,Dropout, Activation, Flatten, Conv2D, MaxPooling2D,BatchNormalization
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
import os 




/home/thanawit/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/thanawit/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
#initialte the variable

batch_size = 32
num_classes = 10
epochs = 100
data_augmentation  = True
num_predictions = 20 
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

In [3]:
#downloading the dataset

(x_train , y_train) , (x_test , y_test ) = cifar10.load_data()

In [4]:
#convert class to binary class

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test =  keras.utils.to_categorical(y_test, num_classes)

In [5]:
x_train.shape[1:]

(32, 32, 3)

In [6]:
#Initialize Layer

model = Sequential()
#Conv2D Filter = filter size, kernel size, stride , padding , format , dilation rate , activation , biased, initializer, bias initilizaer, more 

#Block 1 
model.add(Conv2D( 32 , (3,3) , padding='same',input_shape = x_train.shape[1:]  ))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(32 , (3,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

#Block 2 
model.add(Conv2D(64, (3,3) , padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(64 , (3,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

#Connecting it to Classification
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(num_classes))
model.add(Activation('softmax'))



In [7]:

def lr_schedule(epoch):
    """Learning Rate Schedule
    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.
    # Arguments
        epoch (int): The number of epochs
    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 150:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [8]:
#initiate training lost 
opt = keras.optimizers.rmsprop(lr_schedule(0), decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

#meaning all the result
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

Learning rate:  0.001


In [9]:
#Subtract mean to make it more normalize
x_train_mean = np.mean(x_train, axis=0)
x_train -= x_train_mean
x_test -= x_train_mean

In [10]:
#Decaying LR

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [ lr_reducer, lr_scheduler]




datagen = ImageDataGenerator(
    # set input mean to 0 over the dataset
    featurewise_center=False,
    # set each sample mean to 0
    samplewise_center=False,
    # divide inputs by std of dataset
    featurewise_std_normalization=False,
    # divide each input by its std
    samplewise_std_normalization=False,
    # apply ZCA whitening
    zca_whitening=False,
    # epsilon for ZCA whitening
    zca_epsilon=1e-06,
    # randomly rotate images in the range (deg 0 to 180)
    rotation_range=0,
    # randomly shift images horizontally
    width_shift_range=0.1,
    # randomly shift images vertically
    height_shift_range=0.1,
    # set range for random shear
    shear_range=0.,
    # set range for random zoom
    zoom_range=0.,
    # set range for random channel shifts
    channel_shift_range=0.,
    # set mode for filling points outside the input boundaries
    fill_mode='nearest',
    # value used for fill_mode = "constant"
    cval=0.,
    # randomly flip images
    horizontal_flip=True,
    # randomly flip images
    vertical_flip=False,
    # set rescaling factor (applied before any other transformation)
    rescale=None,
    # set function that will be applied on each input
    preprocessing_function=None,
    # image data format, either "channels_first" or "channels_last"
    data_format=None)
    

    # Compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)



#train model 
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                        validation_data=(x_test, y_test),
                        epochs=epochs, verbose=1, workers=4,
                        callbacks=callbacks, steps_per_epoch = 50000/batch_size)
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)



Learning rate:  0.001
Epoch 1/100
1563/1562 [==============================] - 13s - loss: 1.6407 - acc: 0.4378 - val_loss: 1.1300 - val_acc: 0.5942
Learning rate:  0.001
Epoch 2/100
1563/1562 [==============================] - 12s - loss: 1.2360 - acc: 0.5743 - val_loss: 0.9405 - val_acc: 0.6700
Learning rate:  0.001
Epoch 3/100
1563/1562 [==============================] - 12s - loss: 1.1161 - acc: 0.6212 - val_loss: 0.8716 - val_acc: 0.6983
Learning rate:  0.001
Epoch 4/100
1563/1562 [==============================] - 12s - loss: 1.0346 - acc: 0.6459 - val_loss: 0.9225 - val_acc: 0.7078
Learning rate:  0.001
Epoch 5/100
1563/1562 [==============================] - 12s - loss: 0.9640 - acc: 0.6677 - val_loss: 0.7102 - val_acc: 0.7535
Learning rate:  0.001
Epoch 6/100
1563/1562 [==============================] - 12s - loss: 0.8995 - acc: 0.6927 - val_loss: 0.7489 - val_acc: 0.7534
Learning rate:  0.001
Epoch 7/100
1563/1562 [==============================] - 12s - loss: 0.8466 - acc: 0

1563/1562 [==============================] - 12s - loss: 0.6048 - acc: 0.8038 - val_loss: 0.4870 - val_acc: 0.8373
Learning rate:  0.001
Epoch 56/100
1563/1562 [==============================] - 12s - loss: 0.6098 - acc: 0.8048 - val_loss: 0.4931 - val_acc: 0.8403
Learning rate:  0.001
Epoch 57/100
1563/1562 [==============================] - 12s - loss: 0.6018 - acc: 0.8046 - val_loss: 0.8323 - val_acc: 0.7621
Learning rate:  0.001
Epoch 58/100
1563/1562 [==============================] - 12s - loss: 0.6082 - acc: 0.8040 - val_loss: 0.5988 - val_acc: 0.8287
Learning rate:  0.001
Epoch 59/100
1563/1562 [==============================] - 12s - loss: 0.6068 - acc: 0.8042 - val_loss: 1.0337 - val_acc: 0.7629
Learning rate:  0.001
Epoch 60/100
1563/1562 [==============================] - 12s - loss: 0.6153 - acc: 0.8020 - val_loss: 0.4593 - val_acc: 0.8518
Learning rate:  0.001
Epoch 61/100
1563/1562 [==============================] - 12s - loss: 0.6086 - acc: 0.8044 - val_loss: 0.5422 - v

In [ ]:
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])